In [ ]:
#### Import libraries ####

In [ ]:
import numpy as npq
import pandas as pd
import yaml

In [ ]:
##### Define functions and import data ####

In [ ]:

with open('../config/config.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    data = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
#Make a dictionary with Path to all datasheets
data_sheets = data['FilePaths']

In [ ]:
#Read in CCS configuration File and make sure it is formatted correctly
df_CCS_share = pd.read_csv("../config/CCS_rates.csv",index_col = 0, )
df_CCS_share.columns = df_CCS_share.columns.astype(int)
df_CCS_share = df_CCS_share.dropna(how = "all")

In [ ]:
#Old code - May be used if there are errors with the emissions factors

#df_emission = pd.read_csv("../data/emmsion_factoers.csv")
#df_emission.columns =  ['REGION','FUEL','Emission Factor(Mt_CO2/PJ)']
#Convert From tonnes of carbon to tonnes of carbon dioxide
#co2_per_carbon =  3.67
#kgGJ_to_MtPJ = 1000000
#df_emission['Emission Factor(Mt_CO2/PJ)'] =  df_emission['Emission Factor(Mt_CO2/PJ)'] *co2_per_carbon/kgGJ_to_MtPJ
#df_emission = df_emission.set_index(["REGION","FUEL"])
#df_emission.to_csv("../config/Emissions_factors.csv")

In [ ]:
# Read in emissions data 

In [ ]:
df_emission = pd.read_csv("../config/Emissions_factors.csv",index_col = [0,1])

In [ ]:
#Set out column order for osemosys sheet

In [ ]:
start_cols = ["SCENARIO",'REGION','TECHNOLOGY','EMISSION','MODE_OF_OPERATION','UNITS','NOTES']

In [ ]:
end_cols =  [*range(2017,2071)]

In [ ]:
all_cols = start_cols + end_cols

In [ ]:
#This function generates a emissions "co2" factor By fuel. 

def demand_emissions(df_data_sheet,Emission_map):
    df_EmissionActivityRatio = df_data_sheet.merge(right =df_emission, left_on= ['REGION',"FUEL"], right_index=True)
    df_EmissionActivityRatio.iloc[:,6:-1] = df_EmissionActivityRatio.iloc[:,6:-1].mul(df_EmissionActivityRatio.loc[:,'Emission Factor(Mt_CO2/PJ)'],axis =0 ) *1000
    #df_EmissionActivityRatio = df_EmissionActivityRatio.set_index("REGION","TECHNOLOGY","FUEL").groupby(["REGION","TECHNOLOGY"]).sum()
    #df_EmissionActivityRatio = df_EmissionActivityRatio.reset_index()
    df_EmissionActivityRatio["EMISSION"] = df_EmissionActivityRatio["FUEL"].astype(str) + "_CO2"
    df_EmissionActivityRatio = df_EmissionActivityRatio.drop(["FUEL",'Emission Factor(Mt_CO2/PJ)'],axis =1 )
    df_EmissionActivityRatio["MODE_OF_OPERATION"] = 1
    df_EmissionActivityRatio["UNITS"] = "Mt CO2 per PJ"
    df_EmissionActivityRatio["NOTES"] = np.nan



    return df_EmissionActivityRatio

In [ ]:
### Calculate emissions factors ###
#Run through Data sheets applying the above function. If the scenario column is not present Add it and include emissions for both scenarios

In [ ]:
df_emissions_activity = pd.DataFrame()

for paths in data_sheets:
    input_activity = pd.read_excel(data_sheets[paths], sheet_name = "InputActivityRatio", header=0)
    print ("reading " + f'{paths} at {data_sheets[paths]}')
    try: 
        EmissionActivityRatio = demand_emissions(input_activity,df_emission)
        EmissionActivityRatio = EmissionActivityRatio[all_cols]
        #EmissionActivityRatio.to_excel(data['Output_location']+ "EmissionActivityRatio" + paths+".xlsx" ,index= False)
    except:
        EmissionActivityRatio = demand_emissions(input_activity,df_emission)
        EmissionActivityRatio["SCENARIO"] = "Net-zero"
        EmissionActivityRatio = EmissionActivityRatio[all_cols]
        df_emissions_activity = pd.concat([df_emissions_activity,EmissionActivityRatio])
        EmissionActivityRatio["SCENARIO"] = "Reference"
        #EmissionActivityRatio.to_excel(data['Output_location']+ "EmissionActivityRatio" + paths+".xlsx" ,index= False)
    df_emissions_activity = pd.concat([df_emissions_activity,EmissionActivityRatio])
    print ("done " + f'{paths}')
print("done")    

In [ ]:
### Make adjustments for technologies that use CCS ####

In [ ]:
df_emissions_activity = df_emissions_activity.reset_index(drop = True)

#Find all strings containing "ccs"
df_CCS = df_emissions_activity[df_emissions_activity["TECHNOLOGY"].str.contains('ccs')]

#Make a new data frame for the captured portion of the emissions
df_captured  = df_emissions_activity.iloc[df_CCS.index].copy()
#Make a new column So you can index by sector
df_captured["SECTOER"] = df_captured.loc[:,"TECHNOLOGY"].str.split("_").str[0]

In [ ]:
#Filter the default ccs factors For each sector or from the technology

df_CCS_share = df_CCS_share.reset_index()
defult_CCS_share = df_CCS_share.set_index("TECHNOLOGY").filter(regex = "_DEF", axis=0)
CCS_share  = df_CCS_share.set_index("TECHNOLOGY").drop(defult_CCS_share.index)

In [ ]:
#Clean up the default factors removing the "_DEF" string and make a new index For the sector
defult_CCS_share = defult_CCS_share.reset_index()
defult_CCS_share["SECTOER"] = defult_CCS_share["TECHNOLOGY"].str.split("_").str[0]
defult_CCS_share = defult_CCS_share.drop("TECHNOLOGY",axis =1 )
defult_CCS_share = defult_CCS_share.set_index("SECTOER")

In [ ]:
#Set the index on the Data frame to include the sectorIndex index

In [ ]:
df_captured = df_captured.set_index(["SCENARIO",'REGION',"SECTOER",'TECHNOLOGY','EMISSION','MODE_OF_OPERATION','UNITS','NOTES'])
#Make sure column values are integers
df_captured.columns = df_captured.columns.astype(int)

In [ ]:
#Apply the default factors to all ccs technologies then drop Technologies that have a specific CCS factor
df_captured_def =df_captured.mul(defult_CCS_share, level = 2).drop(CCS_share.index, level = "TECHNOLOGY", axis = 0)
df_captured_sep=df_captured.mul(CCS_share, level = 3 ).dropna(how = "all")
df_captured = pd.concat([df_captured_sep,df_captured_def]).sort_index()

In [ ]:
#Drop the Sector level
df_captured = df_captured.droplevel(level =2 )

In [ ]:
#Subtract captured emissions from total emissions
df_emissions_activity = df_emissions_activity.set_index(start_cols)
df_captured = df_captured.reset_index().set_index(start_cols)
df_emissions_activity = df_emissions_activity.sub(df_captured,fill_value= 0).dropna(how = "all")

In [ ]:
#Rename captured emissions from CO2 to CCS and reset the indexes in the correct format
df_captured = df_captured.reset_index(level=3)
df_captured.loc[:,"EMISSION"] = df_captured["EMISSION"].str.replace("CO2","CCS",regex = True )
df_captured = df_captured.reset_index().set_index(start_cols)

In [ ]:
#Bring together then Emitted and captured CO2 data frames and reset the indexes
df_emissions_activity = pd.concat([df_emissions_activity,df_captured]).reset_index()

In [ ]:
#Save to excel file
df_emissions_activity.to_excel(data['Output_location']+ "EmissionActivityRatio" + pd.Timestamp("today").strftime("%m-%d-%Y-%S") +".xlsx" ,index= False)